# Libraries & Code Setup

In [ ]:
import os
import pandas as pd
import itertools 


PROJ_ROOT = os.path.dirname(os.path.dirname(os.path.abspath('')))
DATA_DIR = os.path.join(PROJ_ROOT, 'catalogs')
LOCAL_DATA = os.path.join(PROJ_ROOT, 'local_data')
print(DATA_DIR)
# DATA_FILE_NAME = 'HG001.strdust.vcf' # set file name here

c:\Users\garyh\Anshutz\Dashnow_Lab\projects\TR-Benchmarking\catalogs


## Function defs

### Helpers

In [ ]:
def compareString(df_list):
    of = open(os.path.join(DATA_DIR, "comp_data.txt"), "w")

    # loop to len - 1 because it is grabbing i+1
    for i in range(len(df_list) - 1):
        #for row in df_list[i]:
        cur_df1 = df_list[i]
        cur_df2 = df_list[i+1]
        offset = 0
        for j in range(5):
            cur_row1 = cur_df1.iloc[j]
            cur_row2 = cur_df2.iloc[j+offset]

            cur_start1 = int(cur_row1["POS"])
            cur_start2 = int(cur_row2["POS"])
            cur_end1 = int(cur_row1["ENDPOS"])
            cur_end2 = int(cur_row2["ENDPOS"])

            out_str = "\n"

            # check for alignment 
            if cur_start1 > cur_start2 & cur_start1 < cur_end2:
                out_str += "Comparison 1 position < Comp 2 position"

                # get the difference between the 2 positions, and then -1 to compensate for 0 index
                pos_offset = cur_start1 - cur_start2 - 1

                comp_section1 = cur_row1["REF"]
                comp_section2 = cur_row2["REF"][pos_offset:]

            elif cur_start1 < cur_start2 & cur_end1 > cur_start2:
                out_str += "Comparison 1 position < Comp 2 position"

                # get the difference between the 2 positions, and then -1 to compensate for 0 index
                pos_offset = cur_start2 - cur_start1 - 1
                print(pos_offset)

                comp_section1 = cur_row1["REF"][pos_offset:]
                comp_section2 = cur_row2["REF"]

            elif cur_end1 < cur_start2:
                # move current row 1 to the next index 
                pass
            elif cur_start1 > cur_end2:
                # move the current row 2 to the next index
                pass
            else:
                out_str += "Comparison Positions equal"

                comp_section1 = cur_row1["REF"]
                comp_section2 = cur_row2["REF"]
            
            
            # append more info to output string that displays the sequence and it's positions
            out_str += f"\nComparing Seq starting at {cur_start1}:\nString1:\n End Pos:{cur_row1["ENDPOS"]}\n {comp_section1}\nString2:\n End Pos:{cur_row2["ENDPOS"]}\n {comp_section2}\n\n"  
            of.write(out_str)

    of.close()


### CODE FROM GEMINI: PARSE AND FIND RELEVANT QUALITIES

In [ ]:
import os
from itertools import cycle # Useful for a simple rate control, as shown below

DATA_DIR = '.' # Example directory

# Placeholder file names
bed_file = 'ref.txt'
file1 = 'file1.txt'
file2 = 'file2.txt'

# --- 1. Open and Initialize ---
# Open the files. We don't use itertools.zip_longest here.
with open(os.path.join(DATA_DIR, bed_file) , 'r') as ref, \
     open(os.path.join(DATA_DIR, file1) , 'r') as f1, \
     open(os.path.join(DATA_DIR, file2) , 'r') as f2:
    
    # Track the status of each file
    ref_done = False
    f1_done = False
    f2_done = False
    
    # Store the most recently read line from each file
    current_ref_line = None
    current_f1_line = None
    current_f2_line = None
    
    # --- 2. The Main Asynchronous Loop ---
    # Loop continues as long as any file is NOT done
    while not (ref_done and f1_done and f2_done):
        
        # --- Asynchronous Control Logic ---
        # THIS IS WHERE YOU DETERMINE WHICH FILE TO ADVANCE.
        
        # Example: Advance 'ref' and 'f1' every cycle, but 'f2' only every 3rd cycle.
        # This is the "different rates" part you asked for.
        
        # 1. Always try to advance ref (the reference file)
        if not ref_done:
            try:
                current_ref_line = next(ref)
                print(f"REF read: {current_ref_line.strip()}")
            except StopIteration:
                ref_done = True
                print("REF finished.")

        # 2. Always try to advance f1
        if not f1_done:
            try:
                current_f1_line = next(f1)
                print(f"F1 read: {current_f1_line.strip()}")
            except StopIteration:
                f1_done = True
                print("F1 finished.")
                
        # 3. Conditional Advancement for f2 (e.g., only if f1 is ahead of a condition)
        # For simplicity, we'll just advance it every cycle for now, but your logic
        # would go here (e.g., if a coordinate in F1 > a coordinate in F2)
        if not f2_done:
            # Check a condition here (e.g., check if current_f1_line meets a criteria)
            # if current_f1_line and meets_criteria(current_f1_line): 
            try:
                current_f2_line = next(f2)
                print(f"F2 read: {current_f2_line.strip()}")
            except StopIteration:
                f2_done = True
                print("F2 finished.")
                
        print("--- End of Loop Iteration ---")

### main logic

In [ ]:
def mainloop(bed_file, file1, file2):

    heads_parsed = False
    file_list = [bed_file, file1, file2]

    with open(os.path.join(DATA_DIR, bed_file) , 'r') as ref, open(os.path.join(DATA_DIR, file1) , 'r') as f1, open(os.path.join(DATA_DIR, file2) , 'r') as f2:
        file_lines=[]
        for i, line in enumerate(itertools.zip_longest(ref, f1, f2)):
            print(line)

            if not heads_parsed:
                # loop through each file (except the reference) and move past the metadata 
                for file_num in range(1, len(file_list)):
                    # while the current line in the current file still contains meta data
                    while line[file_num].startswith("#"):
                        # save starting line for data handling -PROBABLY WONT NEED FOR FINAL LAUNCH
                        data_line_start = i + 1
                        
                        # move to the next line in the current file
                        next(line[file_num])

 

                # set bool to avoid this loop for rest of iterations
                heads_parsed = True

            # if not metadata, begin comparison process
            if not line.startswith("#"):
                # start index just for data lines -PROBABLY WONT NEED FOR FINAL LAUNCH
                data_i = i - data_line_start    

                #split line by tabs into a list
                line_list = line.split("\t") 
                pos = int(line_list[1])
                ref_len = len(line_list[3])

                # calculate the end position and add it to the end of the row list
                end_pos = pos + ref_len - 1
                line_list.append(end_pos)

                # check position with reference bed file

    #print(df)

    #print(type(file_lines))
    #print(file_lines)

    return df

In [12]:
strdust_df = mainloop("test-isolated-vc-catalog.strglr.bed", "HG001.strdust.vcf", "HG001.strkit.vcf")


('chr1\t10627\t10800\tAGGCGCGCCGCGCCGGCGCAGGCGCAGAG\n', '##fileformat=VCFv4.2\n', '##fileformat=VCFv4.2\n')


AttributeError: 'tuple' object has no attribute 'startswith'

In [6]:
strkit_df.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,hg001,ENDPOS
0,chr1,10623,locus1,TGCAAAGGCGCGCCGCGCCGGCGCAGGCGCAGAGAGGCGCGCCGCG...,CGCAGGCGCAGAGAGGCGCGCCGCGCCGGCGCAGGCGCAGAGAGGC...,.,.,VT=str;MOTIF=AGGCGCGCCGCGCCGGCGCAGGCGCAGAG;REF...,GT:PM:DP:MMAS:DPS:AD:MC:MCCI:ANCL:CONS:MCRL,"1/2:dist:45:1.92941:45:37,8:9,12:9-9,9-12:5,12...",10800
1,chr1,103260,locus2,GTAAAATAAAATAAAATAAAA,.,.,.,VT=str;MOTIF=TAAAA;REFMC=4;BED_START=103260;BE...,GT:PM:DP:MMAS:DPS:AD:MC:MCCI:ANCL:CONS:MCRL,"0/0:dist:14:1.99911:14:14:4,4:4-4,4-4:1:.:4x14\n",103280
2,chr1,668334,locus5,GCAAACACGTGGATACATGGAGGGGAACAACACACACCAGGGCCTC...,.,.,.,VT=str;MOTIF=CAAACACGTGGATACATGGAGGGGAACAACACA...,GT:PM:DP:MMAS:DPS:AD:MC:MCCI:ANCL:CONS:MCRL,"0/0:dist:30:1.92977:30:30:2,2:2-2,2-2:1:.:2x30\n",668486
3,chr1,862069,locus6,AGATGGATGATGGATGGATG,.,.,.,VT=str;MOTIF=GATG;REFMC=5;BED_START=862076;BED...,GT:PM:DP:MMAS:DPS:AD:MC:MCCI:ANCL:CONS:MCRL,"0/0:dist:29:1.88007:29:29:5,5:5-5,5-5:1:.:5x29\n",862088
4,chr1,905012,locus7,TGCGGGGGAGGCTGTTGGGGACGTTCGTGGCGGGGGAGGCTGTTGG...,.,.,.,VT=str;MOTIF=GCGGGGGAGGCTGTTGGGGACGTTCGTG;REFM...,GT:PM:DP:MMAS:DPS:AD:MC:MCCI:ANCL:CONS:MCRL,"0/0:dist:26:1.74823:26:26:4,4:4-4,4-4:1:.:4x26\n",905120


In [7]:
strdust_df[strdust_df["POS"] == "10627"]["REF"]


0    AAGGCGCGCCGCGCCGGCGCAGGCGCAGAGAGGCGCGCCGCGCCGG...
Name: REF, dtype: object

In [ ]:
def compareString(df_list):
    of = open(os.path.join(DATA_DIR, "comp_data.txt"), "w")

    # loop to len - 1 because it is grabbing i+1
    for i in range(len(df_list) - 1):
        #for row in df_list[i]:
        cur_df1 = df_list[i]
        cur_df2 = df_list[i+1]
        offset = 0
        for j in range(5):
            cur_row1 = cur_df1.iloc[j]
            cur_row2 = cur_df2.iloc[j+offset]

            cur_start1 = int(cur_row1["POS"])
            cur_start2 = int(cur_row2["POS"])
            cur_end1 = int(cur_row1["ENDPOS"])
            cur_end2 = int(cur_row2["ENDPOS"])

            out_str = "\n"

            # check for alignment 
            if cur_start1 > cur_start2 & cur_start1 < cur_end2:
                out_str += "Comparison 1 position < Comp 2 position"

                # get the difference between the 2 positions, and then -1 to compensate for 0 index
                pos_offset = cur_start1 - cur_start2 - 1

                comp_section1 = cur_row1["REF"]
                comp_section2 = cur_row2["REF"][pos_offset:]

            elif cur_start1 < cur_start2 & cur_end1 > cur_start2:
                out_str += "Comparison 1 position < Comp 2 position"

                # get the difference between the 2 positions, and then -1 to compensate for 0 index
                pos_offset = cur_start2 - cur_start1 - 1
                print(pos_offset)

                comp_section1 = cur_row1["REF"][pos_offset:]
                comp_section2 = cur_row2["REF"]

            elif cur_end1 < cur_start2:
                # move current row 1 to the next index 
                pass
            elif cur_start1 > cur_end2:
                # move the current row 2 to the next index
                pass
            else:
                out_str += "Comparison Positions equal"

                comp_section1 = cur_row1["REF"]
                comp_section2 = cur_row2["REF"]
            
            
            # append more info to output string that displays the sequence and it's positions
            out_str += f"\nComparing Seq starting at {cur_start1}:\nString1:\n End Pos:{cur_row1["ENDPOS"]}\n {comp_section1}\nString2:\n End Pos:{cur_row2["ENDPOS"]}\n {comp_section2}\n\n"  
            of.write(out_str)

    of.close()


In [25]:
compareString([strdust_df, strkit_df])

# Resources

File bunching: https://stackoverflow.com/questions/16669428/process-very-large-20gb-text-file-line-by-line
reading in vcf to dataframe: https://gist.github.com/dceoy/99d976a2c01e7f0ba1c813778f9db744